In [8]:
from extractor import get_text
from scan import get_files
import pandas as pd
import os
import json
from prompt import *
from pathlib import Path
from tqdm.auto import tqdm
import requests


In [9]:
PATH = r"C:\Users\alika\OneDrive\Masaüstü\Profiler\FileProfiler\test\wiki_docs"

In [10]:
directory = Path(PATH)
rows = []
for f in directory.iterdir():
    if f.is_file():
        rows.append({
        "Full Path": str(f.resolve()),
        "File Name": f.stem,
        "Extension": f.suffix.lstrip('.')
        })

df = pd.DataFrame(rows).head(10)

In [11]:
text = get_text(df['Full Path'][0])
question = r'Is there any personal information in the document ?'

In [12]:
headers = {"Authorization": f"Bearer {os.getenv("OPENROUTER_KEY")}",
            "Content-Type": "application/json",}

data = {"model": "qwen/qwen3-8b",
        "messages": [{"role": "user", "content": sp},
                        {"role": "user", "content": up.format(text, question)}],}

r = requests.post(os.getenv("OPENROUTER_URL"),
                headers=headers, data=json.dumps(data))
if r.status_code==200:
        resp = json.loads(r.text.strip())["choices"][0]["message"]["content"]